In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from category_encoders import TargetEncoder
from category_encoders import OneHotEncoder
from category_encoders import CountEncoder
from category_encoders import OrdinalEncoder
import random
from math import floor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

# Praca domowa 2
**Mikołaj Spytek**

W tej pracy domowej zajmuję się zbiorem danych Allegro. W pierwszej części chodzi o kodowanie zmiennych kategorycznych, natomiast w drugiej o preprocessing - uzupełnianie danych brakujących.

## Część pierwsza

W pierwszej części kodujemy zmienne kategoryczne - zmienną `it_location`  za pomocą target encoding oraz zmienną `main_category` za pomocą one-hot encodingu oraz dwóch innych metod.

In [ ]:
# wczytanie zbioru danych
df = pd.read_csv("https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1")
# zauważyłem, ze nazwy lokalizacji są różnie pisane, więc ujednolicam
df[["it_location"]] = df[["it_location"]].apply(lambda x: x.str.lower())

#stworzenie kopii, aby każdy typ encodingu był na osobnej ramce
dftarget = df.copy()
dfonehot = df.copy()
dfcount = df.copy()
dfordinal = df.copy()
df_part2 = df[["price", "it_seller_rating", "it_quantity"]]

In [ ]:
df[["it_location"]].describe()

Gdy spojrzymy na zmienną, którą chcemy zakodować, zauważamy, że przyjmuje ona 7903 różnych wartości. Gdybyśmy więc chieli zastosować one-hot encoding, to powstałoby właśnie tyle nowych kolumn, czyli zmiennych. Ponieważ mamy dużo obserwacji - nadal byłoby około rząd wielkości więcej niż zmiennych, to mogłoby się okazać, że model będzie działać, lecz target encoding jest sposobem na zmniejszenie ilości zmiennych objaśniających.

In [ ]:
# target encoding zmiennej it_location
en = TargetEncoder()
dftarget["target_encoded_it_location"] = en.fit_transform(dftarget["it_location"], dftarget["price"])
# sprawdzenie czy pojawiła się nowa kolumna
dftarget.head()

Widzimy, że w ramce danych pojawiła się nowa kolumna: `target_encoded_it_location`. Jest to zmienna zawierająca zakodowane wartości kategorii.

Target encoding polega na pogrupowaniu danych według kategorii, a następnie wyliczenia dla każdej z tych kategorii średniej wartości zmiennej wyjaśnianej. Otrzymaną w ten sposób wartością kodujemy wszystkie obserwacje z danej kategorii.

Na przykładzie można sprawdzić, czy ten Encoder rzeczywiście tak działa. Dla Warszawy policzymy średnią cenę produktu "ręcznie", a następnie sprawdzimy czy taka sama wartość została nadana przez Encoder.

In [ ]:
print("Średnia wyliczona ręcznie: ", dftarget.loc[dftarget["it_location"]=="warszawa", "price"].mean())
print("Wartość z encodingu: ", dftarget.loc[dftarget["it_location"]=="warszawa", "target_encoded_it_location"].head(1))

Wygląda na to, że wszystko się zgadza.

#### Jakie są wady takiego kodowania?
- zamieniając zmienną kategoryczną na tylko jedną zmienną numeryczną wprowadzamy porządek (możliwość porównania) kategorii, który wcześniej nie istniał, 
- jeśli w którejś kategorii było mało obserwacji, to średnia wartość targetu może być w łatwy sposób zaburzona, 
- dane zakodowane w ten sposób mogą powodować przeuczenie się modelu

In [ ]:
df[["main_category"]].describe()

Widzimy, że ta zmienna przyjmuje już tylko 27 unikalnych wartości, więc zastosowanie one-hot encodingu ma tu dużo większy sens.

In [ ]:
en = OneHotEncoder(use_cat_names=True)

dfonehot = dfonehot.join(en.fit_transform(dfonehot["main_category"]))
dfonehot.head()

Widzimy, że teraz ramka danych ma 41, kolumny, podczas gdy wcześniej miała 14. Oznacza to, że OneHotEncoder dodał 27 nowych zmiennych - dokładnie tyle ile było unikalnych wartości kolumny `main_category`. Jest to zarówno zaleta, jak i wada tego rodzaju kodowania. Dzięki temu, że każda kategoria ma osobną zmienną, możemy mieć pewność, że nie będą na siebie wpływały w modelu, tak jak to może być w przypadku kodowań, które tworzą tylko jedną zmienną. 

In [ ]:
en = CountEncoder()

dfcount["count_encoded_main_category"] = en.fit_transform(dfcount["main_category"])

dfcount.head()

W wyniku zastosowania tego kodowania, każdej obserwacji przypisywana jest liczba produktów w danej kategorii. Mamy więc tylko jedną dodatkową kolumnę. W ramach sprawdzenia można popatrzeć na wiersz 2 z powyższej ramki - obserwacja z kategorii `Dom i ogród` została zakodowana jako 91042, a wcześniej gdy sprawdzaliśmy liczbę kategorii, polecenie `df["main_category"].describe()` pokazało, że najczęstszą kategorią jest właśnie ta i pojawia się 91042 razy w tym zbiorze danych.

Wadą tego typu encodingu również jest to, że kategoriom nadajemy arbitralny porządek. Dodatkowo kategorie o podobnej liczności będą miały podobną wartość zmiennej kodującej kategorie, a mogą być one zupełnie różny wpływ na zmienną wyjaśnianą.


In [ ]:
en = OrdinalEncoder()

dfordinal["ordinal_encoded_main_category"] = en.fit_transform(dfordinal["main_category"])

dfordinal.head()

Ta metoda jest bardzo naiwna, i często pociąga niepożądane konsekwencje. Enkoder po prostu przyporządkowuje kategoriom kolejne liczby naturalne. Znów problem pojawia się, ponieważ dostajemy jakiś porządek na kategoriach, którego wcześniej nie było. Model może więc się nauczyć nieistniejących zależności. 

## Część druga

Wypełnianie braków w zmiennych.

In [ ]:
df_part2.head()

In [ ]:
# ustawiamy seed, aby wyniki były powtarzalne
random.seed(123)
# wybieramy podzbiór kolumn
df_part2 = df[["price", "it_seller_rating", "it_quantity"]]

imputer = KNNImputer(weights="uniform", n_neighbors=5)

#ograniczmy liczbę rekordów - przy pełnej ramce obliczenia wykonywały się bardzo długo
df_part2 = df_part2[:floor(len(df_part2)*0.1)]
rmse = [None for i in range(10)]

type(df_part2)
len(df_part2)
for i in range(10):
    # robimy kopię ramki
    df_part2copy = df_part2.copy()
    # wybieramy 10% losowych indeksów
    toberemoved = random.sample(range(0, len(df_part2)), floor(len(df_part2)*0.1))
    # usuwamy
    df_part2copy.iloc[toberemoved, 1] = None
    # imputacja
    df_part2copy = pd.DataFrame(imputer.fit_transform(df_part2copy), columns=["price", "it_seller_rating", "it_quantity"])

    # wyliczenie błędu
    rmse[i] = np.sqrt(mean_squared_error(df_part2["it_seller_rating"], df_part2copy["it_seller_rating"]))
    print("RMSE {:.2f}".format(rmse[i]))
    
    
print("Odchylenie standardowe miary RMSE w 10 próbach wynosi: {:.2f}".format(np.std(rmse)))


Powtarzamy ten sam eksperyment tylko, że usuwamy wartości z dwóch kolumn

In [ ]:
df_part2 = df[["price", "it_seller_rating", "it_quantity"]]

imputer = KNNImputer(weights="uniform", n_neighbors=5)

#ograniczmy liczbę rekordów
df_part2 = df_part2[:floor(len(df_part2)*0.1)]
rmse1 = [None for i in range(10)]
rmse2 = [None for i in range(10)]


type(df_part2)
len(df_part2)
for i in range(10):
    df_part2copy = df_part2.copy()
    toberemoved = random.sample(range(0, len(df_part2)), floor(len(df_part2)*0.1))
    toberemoved2 = random.sample(range(0, len(df_part2)), floor(len(df_part2)*0.1))
    df_part2copy.iloc[toberemoved, 1] = None
    df_part2copy.iloc[toberemoved, 2] = None

    
    df_part2copy = pd.DataFrame(imputer.fit_transform(df_part2copy), columns=["price", "it_seller_rating", "it_quantity"])

    
    rmse1[i] = np.sqrt(mean_squared_error(df_part2["it_seller_rating"], df_part2copy["it_seller_rating"]))
    rmse2[i] = np.sqrt(mean_squared_error(df_part2["it_quantity"], df_part2copy["it_quantity"]))

    print("RMSE kolumny it_seller_rating: {:.2f}".format(rmse1[i]))
    print("RMSE kolumny it_quantity: {:.2f}".format(rmse2[i]))
    
    
    
#print("Odchylenie standardowe miary RMSE kolumny it_seller_rating w 10 próbach wynosi: {:.2f}".format(np.std(rmse1)))
#print("Odchylenie standardowe miary RMSE kolumny it_quantity w 10 próbach wynosi: {:.2f}".format(np.std(rmse2)))

d = {"RMSE kolumny it_seller_rating": rmse1, "Rmse kolumny it_quantity":rmse2 }
results = pd.DataFrame(d)
results

In [ ]:
np.std(results)

In [ ]:
fig = plt.figure(figsize=(12,12))
p = plt.plot([i for i in range(1, 11)], rmse, [i for i in range(1, 11)], rmse1)
plt.ylim([0, 13000])
plt.legend(["Usunięcie tylko it_seller_rating", "Usunięcie obu kolumn"])
plt.title("RMSE dla kolumny it_seller_rating")
plt.xlabel("numer iteracji")
plt.ylabel("RMSE")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.boxplot(data=results)
plt.ylim([0,13000])

Z powyższych wykresów widać, że jeśli usuniemy dane z więcej niż jednej kolumny, algorytm imputacji działa gorzej. Może to być spowodowane, że w pierwszym przypadku znajduje "lepszych", tzn. bardziej powiązanych sąsiadów. 

Innym czynnikiem, który może mieć wpływ na wynik, jest użyta metryka. W tym przypadku używam domyślnej metryki euklidesowej.